# Inspecting SageMaker Debugger Logs and Results 

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r artifacts_path
artifacts_path

In [ ]:
!pip install smdebug

In [ ]:
from smdebug.trials import create_trial

trial = create_trial(artifacts_path)

In [ ]:
trial.tensor_names()

In [ ]:
trial.__dict__

In [ ]:
trial.__dict__.keys()

In [ ]:
trial.bucket_name

In [ ]:
trial.prefix_name

In [ ]:
trial.prefix_name

In [ ]:
target_path = f"s3://{trial.bucket_name}/{trial.prefix_name}"
target_path

In [ ]:
s3_contents = !aws s3 ls {target_path} --recursive
s3_contents

In [ ]:
chosen_path = s3_contents[-1].split(" ")[-1]
chosen_path

In [ ]:
chosen_fullpath = f"s3://{trial.bucket_name}/{chosen_path}"
chosen_fullpath

In [ ]:
!mkdir -p tmp

In [ ]:
!aws s3 cp {chosen_fullpath} tmp/worker_0.json

In [ ]:
!cat tmp/worker_0.json

In [ ]:
trial.tensor("train-error").workers(2)

In [ ]:
for i in [2, 4, 6, 8, 10, 12, 14, 16]:
    traint = trial.tensor("train-error")
    train_error = traint.value(i)[0]
    train_error = "{0:.4f}".format(train_error)
    
    valt = trial.tensor("validation-error")
    validation_error = valt.value(i)[0]
    validation_error = "{0:.4f}".format(validation_error)
    print(f"STEP {i}: [TRAIN ERROR]={train_error} " +
          "[VALIDATION ERROR]={validation_error}")

In [ ]:
%store -r rule_job_summary

In [ ]:
def lfx(r):
    return r["RuleConfigurationName"] == "LossNotDecreasing"

loss_not_decreasing_summary = list(filter(
    lfx, 
    rule_job_summary))[0]

loss_not_decreasing_summary

In [ ]:
summary = loss_not_decreasing_summary
rule_evaluation_job_arn = summary['RuleEvaluationJobArn']
rule_evaluation_job_arn

In [ ]:
import sagemaker 
from sagemaker.processing import ProcessingJob

session = sagemaker.Session()
processing_job = ProcessingJob.from_processing_arn(
    sagemaker_session=session, 
    processing_job_arn=rule_evaluation_job_arn)

processing_job

In [ ]:
processing_job.__dict__

In [ ]:
processing_job.outputs[0].__dict__

In [ ]:
processing_job.job_name

In [ ]:
region = "us-east-1"
group = "/aws/sagemaker/ProcessingJobs"
prefix = processing_job.job_name

In [ ]:
!pip install awslogs

In [ ]:
!awslogs get {group} --log-stream-name-prefix {prefix} -s3h --aws-region {region}